In [1]:
%pip install rich smdebug_rulesconfig schema

Note: you may need to restart the kernel to use updated packages.


In [2]:
from sagemaker.estimator import Estimator
from sagemaker.session import Session
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
role = get_execution_role()
sess = Session()

In [4]:
estimator = Estimator(
    image_uri='155954279556.dkr.ecr.us-east-1.amazonaws.com/gs-automl-base-containers/boilerplate312_sm:1.0',
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    hyperparameters={
        "table_name": "automl-classification-experiment",
        "project_hashkey": "2ee07a49",
        "experiment_hashkey": "1cbd8309",
        "dataset_table_name": "automl-dataset",
        "dataset_profile_table_name": "automl-dataset-profile-experiment-result",
        "model_repo_table_name": "automl-model-repo",
        "model_experiment_result_table_name": "automl-classification-experiment",
        "username": "sean@gs.co.kr",
        "job_type": "training",
        "task_token": "1234",
    },
    base_job_name='custom-training',
    sagemaker_session=sess
)

estimator.fit()

INFO:sagemaker:Creating training-job with name: custom-training-2026-03-01-11-33-59-660


2026-03-01 11:34:01 Starting - Starting the training job...
2026-03-01 11:34:17 Starting - Preparing the instances for training...
2026-03-01 11:34:40 Downloading - Downloading input data...
2026-03-01 11:35:06 Downloading - Downloading the training image..2026-03-01 11:35:45,992 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2026-03-01 11:35:45,992 sagemaker-training-toolkit INFO     Failed to parse hyperparameter dataset_profile_table_name value automl-dataset-profile-experiment-result to Json.
Returning the value itself
2026-03-01 11:35:45,993 sagemaker-training-toolkit INFO     Failed to parse hyperparameter dataset_table_name value automl-dataset to Json.
Returning the value itself
2026-03-01 11:35:45,993 sagemaker-training-toolkit INFO     Failed to parse hyperparameter experiment_hashkey value 1cbd8309 to Json.
Returning the value itself
2026-03-01 11:35:45,993 sagemaker-training-toolkit INFO     Failed to parse hyperparameter job_type value t

In [5]:
# 이후, 최신 트레이닝 잡 정보 접근
training_job_name = estimator.latest_training_job.name
print("Training Job Name:", training_job_name)

# CloudWatch 로그 그룹과 스트림 이름 가져오기
log_group = f"/aws/sagemaker/TrainingJobs"
log_stream = f"{training_job_name}/algo-1"

print("Log Group:", log_group)
print("Log Stream:", log_stream)

Training Job Name: custom-training-2026-03-01-11-33-59-660
Log Group: /aws/sagemaker/TrainingJobs
Log Stream: custom-training-2026-03-01-11-33-59-660/algo-1


In [6]:
import boto3

logs_client = boto3.client("logs", region_name="ap-northeast-2")

In [7]:
streams = logs_client.describe_log_streams(
    logGroupName="/aws/sagemaker/TrainingJobs",
    logStreamNamePrefix=f"{training_job_name}/algo-1"
)

for stream in streams['logStreams']:
    print(stream['logStreamName'])  # 전체 UUID 포함된 스트림 이름


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 streams = logs_client.describe_log_streams(                                                  │
│   2 │   logGroupName="/aws/sagemaker/TrainingJobs",                                              │
│   3 │   logStreamNamePrefix=f"{training_job_name}/algo-1"                                        │
│   4 )                                                                                            │
│                                                                                                  │
│ /home/ec2-user/SageMaker/.myenv/miniconda/envs/boilerplate312/lib/python3.12/site-packages/botoc │
│ ore/client.py:602 in _api_call                                                                   │
│                                                                                                  │
│    599 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    600 │   │   │   │   )                                                                         │
│    601 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  602 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    603 │   │                                                                                     │
│    604 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    605                                                                                           │
│                                                                                                  │
│ /home/ec2-user/SageMaker/.myenv/miniconda/envs/boilerplate312/lib/python3.12/site-packages/botoc │
│ ore/context.py:123 in wrapper                                                                    │
│                                                                                                  │
│   120 │   │   │   with start_as_current_context():                                               │
│   121 │   │   │   │   if hook:                                                                   │
│   122 │   │   │   │   │   hook()                                                                 │
│ ❱ 123 │   │   │   │   return func(*args, **kwargs)                                               │
│   124 │   │                                                                                      │
│   125 │   │   return wrapper                                                                     │
│   126                                                                                            │
│                                                                                                  │
│ /home/ec2-user/SageMaker/.myenv/miniconda/envs/boilerplate312/lib/python3.12/site-packages/botoc │
│ ore/client.py:1078 in _make_api_call                                                             │
│                                                                                                  │
│   1075 │   │   │   │   'error_code_override'                                                     │
│   1076 │   │   │   ) or error_info.get("Code")                                                   │
│   1077 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1078 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1079 │   │   else:                                                                             │
│   1080 │   │   │   return parsed_response                                                        │
│   1081                                                     